Siamese_tpxtech　テスト

In [1]:
import os
import zipfile
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import seaborn as sns

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


In [3]:
BATCH_SIZE = 64

def get_dataset_slice_paths(image_dir, k):
  '''
  画像のパスリストを返す
  '''
  image_file_list = os.listdir(image_dir + k)
  image_paths = [os.path.join(image_dir+k, fname) for fname in image_file_list]

  return image_paths


In [4]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [5]:
from PIL import Image
import numpy as np

class_word = ['dust', 'friction', 'hair', 'pinhole']
train_data = []
train_label = []

for i, k in enumerate(class_word):
    training_image_paths = get_dataset_slice_paths('./tpxtech/', k)
    # 画像ペアを6チャンネルとして合わせる
    cnt = 0
    for j in range(0, len(training_image_paths), 2):
        im = np.array(Image.open(training_image_paths[j]))
        im2 = np.array(Image.open(training_image_paths[j + 1]))
        im_concat = np.concatenate([im, im2], 2)
        im_concat = im_concat.tolist()
        train_data.append(im_concat)
        cnt += 1
    train_label.extend([i for m in range(cnt)])


In [6]:
# train_data = np.array(train_data)
# train_label = np.array(train_label)
# train_data.shape, train_label.shape
train_data = np.array(train_data)
train_label = np.array(train_label)
train_data.shape,  train_label.shape

((952, 80, 80, 6), (952,))

In [7]:
# ペアの生成

def make_pairs(images, labels):
    pairImage = []
    pairLabel = []
    CLASS = 4
    label_idx = [np.where(labels == i)[0] for i in range(CLASS)]
    
    for idx in range(len(images)):
        current = images[idx]
        label = labels[idx]
        
        # ポジティブ
        for i in range(2):
          idx_posi = np.random.choice(label_idx[label])
          img_posi = images[idx_posi]
          pairImage.append([current, img_posi])
          pairLabel.append([1])
        
        # ネガティブ
        for j in range(2):
          neg = np.where(labels != label)[0]
          idx_neg = np.random.choice(neg)
          img_neg = images[idx_neg]
          pairImage.append([current, img_neg])
          pairLabel.append([0])

        # # 10C2のペアを作成 => 重いのでペンディング
        # for pair_idx in range(idx+1, len(images)):
        #   pair_img = images[pair_idx]
        #   pair_label = labels[pair_idx]
        #   pairImage.append([current, pair_img])
        #   if label == pair_label:
        #     pairLabel.append([1])
        #   else:
        #     pairLabel.append([0])

        
    # ランダムに並び替える
    per = np.random.permutation(np.arange(len(images)*4))
    pairImage = np.array(pairImage)
    pairImage = pairImage[per]
    
    pairLabel = np.array(pairLabel)
    pairLabel = pairLabel[per]
    
    # テスト、バリデーションに分割して返す
    from sklearn.model_selection import train_test_split
    xData_train, xData_test, yData_train, yData_test = train_test_split(pairImage, pairLabel, test_size=0.2)
    
    return (xData_train, xData_test, yData_train, yData_test)
    
        

In [ ]:
xData_train, xData_test, yData_train, yData_test = make_pairs(train_data, train_label)

In [ ]:
xData_train.shape, xData_test.shape, yData_train.shape, yData_test.shape

In [2]:
# import for model

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Lambda, Conv2D, Dense, Dropout,GlobalAveragePooling2D, MaxPool2D
import tensorflow.keras.backend as K

In [3]:
IMG_SHAPE = (80, 80, 6)

def build_siamese_model(input_shape, output_dim=48):
    inputs = Input(input_shape)
    x = Conv2D(64, (2,2), padding='same', activation='relu')(inputs)
    x = MaxPool2D(pool_size=(2,2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(64, (2,2), padding='same', activation='relu')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    pooled_output = GlobalAveragePooling2D()(x)
    outputs = Dense(output_dim)(pooled_output)
    
    model = Model(inputs, outputs)
    return model

model = build_siamese_model(IMG_SHAPE)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80, 80, 6)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 80, 80, 64)        1600      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 40, 40, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 40, 40, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 64)        16448     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 64)        0     

In [4]:

# １つのインスタンスを共有することでパラメータを共有する


imgA = Input(shape=IMG_SHAPE)
imgB = Input(shape=IMG_SHAPE)

feature_extractor = build_siamese_model(IMG_SHAPE)
modelA = feature_extractor(imgA)
modelB = feature_extractor(imgB)


In [5]:
# ユークリッド距離を計算する関数
# レイヤとして埋め込むためにkerasで関数を生成する

def euclidean_distance(vectors):
    (A, B) = vectors
    sumSquared = K.sum(K.square(A - B), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))


dist = Lambda(euclidean_distance)([modelA, modelB])
outputs = Dense(1, activation='sigmoid')(dist)
model = Model(inputs=[imgA, imgB], outputs=outputs)

In [6]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 80, 80, 6)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 80, 80, 6)]  0                                            
__________________________________________________________________________________________________
model_1 (Functional)            (None, 48)           21168       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda (Lambda)                 (None, 1)            0           model_1[0][0]              

In [14]:
from tensorflow.keras.utils import plot_model

plot_model(model)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [11]:
! brew install graphviz

/bin/bash: brew: command not found


In [ ]:
def contrastive_loss(y, preds, margin=1):
    y = tf.cast(y, preds.dtype)
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y*squaredPreds + (1 - y)*squaredMargin)
    return loss

In [ ]:
# 0に近い方場合同一として判定

def dist_accuracy(y, preds):
    return K.mean(K.equal(y, K.cast(preds < 0.5, y.dtype)))

In [ ]:
# binary_crossentropyの方が簡単に試せる
model.compile(
    loss = contrastive_loss,
    optimizer = 'adam',
    metrics = [dist_accuracy]
)

In [ ]:

def plot_training(history):
    plt.style.use('ggplot')
    plt.figure()
    plt.plot(history.history['loss'], label='train loss')
    plt.plot(history.history['val_loss'], label='val loss')
    plt.plot(history.history['dist_accuracy'], label='train acc')
    plt.plot(history.history['val_dist_accuracy'], label='val_acc')
    plt.legend(loc = 'lower left')

In [ ]:
history = model.fit(
    [xData_train[:,0], xData_train[:,1]], yData_train[:],
    validation_data=([xData_test[:,0], xData_test[:,1]], yData_test[:]),
    batch_size = BATCH_SIZE,
    epochs = 5
)

In [ ]:
xData_train[:,0].shape, yData_train.shape

In [ ]:
yData_test.shape

In [ ]:
plot_training(history)